In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.133718490600586 | KNN Loss: 6.226861000061035 | BCE Loss: 1.9068574905395508
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.137260437011719 | KNN Loss: 6.226616859436035 | BCE Loss: 1.9106433391571045
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.188157081604004 | KNN Loss: 6.226138114929199 | BCE Loss: 1.9620192050933838
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.124198913574219 | KNN Loss: 6.226280212402344 | BCE Loss: 1.8979182243347168
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.152647018432617 | KNN Loss: 6.226075172424316 | BCE Loss: 1.9265716075897217
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.139322280883789 | KNN Loss: 6.2255659103393555 | BCE Loss: 1.913756012916565
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.129941940307617 | KNN Loss: 6.225582599639893 | BCE Loss: 1.9043588638305664
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.077471733093262 | KNN Loss: 6.2255635261535645 | BCE Loss: 1.8519

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.2970476150512695 | KNN Loss: 5.1975531578063965 | BCE Loss: 1.0994946956634521
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 6.278271198272705 | KNN Loss: 5.143256664276123 | BCE Loss: 1.135014533996582
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.123623847961426 | KNN Loss: 5.0088276863098145 | BCE Loss: 1.1147961616516113
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.025326251983643 | KNN Loss: 4.92909574508667 | BCE Loss: 1.0962305068969727
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.927366256713867 | KNN Loss: 4.832589149475098 | BCE Loss: 1.0947771072387695
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.831093788146973 | KNN Loss: 4.722562313079834 | BCE Loss: 1.1085315942764282
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.719992637634277 | KNN Loss: 4.626026630401611 | BCE Loss: 1.0939661264419556
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.601950168609619 | KNN Loss: 4.527592182159424 | BCE Los

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.895739555358887 | KNN Loss: 3.8490068912506104 | BCE Loss: 1.046732783317566
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.902154922485352 | KNN Loss: 3.8195180892944336 | BCE Loss: 1.082637071609497
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.926777362823486 | KNN Loss: 3.8551180362701416 | BCE Loss: 1.0716592073440552
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.889044284820557 | KNN Loss: 3.8556411266326904 | BCE Loss: 1.0334031581878662
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.888866901397705 | KNN Loss: 3.8489649295806885 | BCE Loss: 1.0399020910263062
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.8839311599731445 | KNN Loss: 3.8232359886169434 | BCE Loss: 1.0606951713562012
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.8758344650268555 | KNN Loss: 3.8018832206726074 | BCE Loss: 1.0739511251449585
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.916973114013672 | KNN Loss: 3.8316996097564697 |

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.830670356750488 | KNN Loss: 3.7763350009918213 | BCE Loss: 1.0543352365493774
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.85447883605957 | KNN Loss: 3.7854175567626953 | BCE Loss: 1.069061279296875
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.841700077056885 | KNN Loss: 3.8065178394317627 | BCE Loss: 1.035182237625122
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.8088531494140625 | KNN Loss: 3.7778830528259277 | BCE Loss: 1.0309698581695557
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.791323184967041 | KNN Loss: 3.7804763317108154 | BCE Loss: 1.0108469724655151
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.834181785583496 | KNN Loss: 3.7814431190490723 | BCE Loss: 1.0527385473251343
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.873087406158447 | KNN Loss: 3.8162765502929688 | BCE Loss: 1.056810736656189
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.836986541748047 | KNN Loss: 3.7850193977355957 | BC

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.8183817863464355 | KNN Loss: 3.7670352458953857 | BCE Loss: 1.0513464212417603
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.799330711364746 | KNN Loss: 3.7650609016418457 | BCE Loss: 1.0342698097229004
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.820438861846924 | KNN Loss: 3.7949063777923584 | BCE Loss: 1.0255324840545654
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.801769733428955 | KNN Loss: 3.762892723083496 | BCE Loss: 1.038877010345459
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.841640472412109 | KNN Loss: 3.7723915576934814 | BCE Loss: 1.0692486763000488
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.806844711303711 | KNN Loss: 3.7636656761169434 | BCE Loss: 1.0431790351867676
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.792999267578125 | KNN Loss: 3.770460605621338 | BCE Loss: 1.0225389003753662
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.766674041748047 | KNN Loss: 3.7495651245117188 | B

...

Epoch 448 / 500 | iteration 10 / 30 | Total Loss: 4.7324347496032715 | KNN Loss: 3.7232372760772705 | BCE Loss: 1.009197473526001
Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.746771812438965 | KNN Loss: 3.7235639095306396 | BCE Loss: 1.0232077836990356
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.696274280548096 | KNN Loss: 3.687047004699707 | BCE Loss: 1.0092272758483887
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.720311641693115 | KNN Loss: 3.698303699493408 | BCE Loss: 1.0220078229904175
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.745702743530273 | KNN Loss: 3.7228331565856934 | BCE Loss: 1.022869348526001
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.699822425842285 | KNN Loss: 3.6966190338134766 | BCE Loss: 1.0032036304473877
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.670045375823975 | KNN Loss: 3.6735451221466064 | BCE Loss: 0.9965003728866577
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 4.763278961181641 | KNN Loss: 3.7159793376922

Epoch 458 / 500 | iteration 25 / 30 | Total Loss: 4.690435409545898 | KNN Loss: 3.6791489124298096 | BCE Loss: 1.011286735534668
Epoch 459 / 500 | iteration 0 / 30 | Total Loss: 4.689745903015137 | KNN Loss: 3.6812632083892822 | BCE Loss: 1.0084829330444336
Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.755011558532715 | KNN Loss: 3.717414140701294 | BCE Loss: 1.037597417831421
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.73530387878418 | KNN Loss: 3.7053306102752686 | BCE Loss: 1.0299732685089111
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.7699384689331055 | KNN Loss: 3.739673614501953 | BCE Loss: 1.0302648544311523
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.737393379211426 | KNN Loss: 3.72377610206604 | BCE Loss: 1.0136171579360962
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.752344131469727 | KNN Loss: 3.733727216720581 | BCE Loss: 1.0186166763305664
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.694576740264893 | KNN Loss: 3.697068929672241 | 

Epoch 469 / 500 | iteration 15 / 30 | Total Loss: 4.790313720703125 | KNN Loss: 3.750507116317749 | BCE Loss: 1.0398067235946655
Epoch 469 / 500 | iteration 20 / 30 | Total Loss: 4.7207794189453125 | KNN Loss: 3.701083183288574 | BCE Loss: 1.0196962356567383
Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.785818099975586 | KNN Loss: 3.729259490966797 | BCE Loss: 1.0565588474273682
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.736263275146484 | KNN Loss: 3.694779872894287 | BCE Loss: 1.0414836406707764
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.717832565307617 | KNN Loss: 3.720715284347534 | BCE Loss: 0.9971174597740173
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.718566417694092 | KNN Loss: 3.7001214027404785 | BCE Loss: 1.0184450149536133
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.682604789733887 | KNN Loss: 3.677612781524658 | BCE Loss: 1.004992127418518
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.745893478393555 | KNN Loss: 3.726593255996704 

Epoch 480 / 500 | iteration 5 / 30 | Total Loss: 4.699853897094727 | KNN Loss: 3.69313383102417 | BCE Loss: 1.0067198276519775
Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.660924911499023 | KNN Loss: 3.6677608489990234 | BCE Loss: 0.9931640625
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.731444358825684 | KNN Loss: 3.7100045680999756 | BCE Loss: 1.021439552307129
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.725133419036865 | KNN Loss: 3.7023613452911377 | BCE Loss: 1.0227720737457275
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.721109390258789 | KNN Loss: 3.695420742034912 | BCE Loss: 1.0256887674331665
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.707658767700195 | KNN Loss: 3.6958277225494385 | BCE Loss: 1.011831283569336
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.735317230224609 | KNN Loss: 3.6926376819610596 | BCE Loss: 1.0426793098449707
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.760739803314209 | KNN Loss: 3.712620496749878 | BCE 

Epoch 490 / 500 | iteration 25 / 30 | Total Loss: 4.691200256347656 | KNN Loss: 3.7076048851013184 | BCE Loss: 0.9835952520370483
Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.730219841003418 | KNN Loss: 3.7243521213531494 | BCE Loss: 1.0058677196502686
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.709720134735107 | KNN Loss: 3.6959407329559326 | BCE Loss: 1.0137794017791748
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.751674175262451 | KNN Loss: 3.746224880218506 | BCE Loss: 1.0054491758346558
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.745136260986328 | KNN Loss: 3.7450082302093506 | BCE Loss: 1.0001280307769775
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.755199432373047 | KNN Loss: 3.7198448181152344 | BCE Loss: 1.0353548526763916
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.71691370010376 | KNN Loss: 3.7100090980529785 | BCE Loss: 1.0069046020507812
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.70306921005249 | KNN Loss: 3.67787456512451

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.0376,  4.0441,  2.9325,  2.2331,  2.3031,  0.7054,  2.1160,  1.6226,
          2.6057,  1.9758,  1.6559,  1.7841,  0.4641,  1.8709,  1.3173,  1.0579,
          3.1501,  2.2467,  3.1790,  2.6883,  1.7776,  2.4152,  1.8910,  3.0578,
          2.8368,  1.8914,  2.1541,  1.4339,  1.7862,  0.6679, -0.0071,  0.7831,
          0.5321,  0.9999,  1.7067,  1.2602,  1.1641,  3.7554,  1.0405,  1.4261,
          0.7441, -1.0527, -0.0267,  2.4635,  2.0952,  0.8401, -0.3758,  0.2380,
          1.6674,  2.5574,  1.9121,  0.4660,  1.3182,  0.2007, -0.6321,  1.3695,
          1.4160,  1.5192,  1.3526,  1.9552,  0.1368,  1.1053,  0.3451,  1.2740,
          1.1374,  2.0587, -1.5341,  0.5228,  2.6743,  1.9066,  2.9133,  0.1733,
          1.4021,  2.7667,  1.9263,  1.1434,  0.5951,  0.6298,  0.0815,  1.9243,
          0.1050,  0.4397,  1.9906, -0.2688,  0.1968, -1.3471, -2.4413, -0.0647,
          0.7598, -1.8507,  0.6403,  0.1251, -0.5934, -1.3275,  0.4164,  1.4477,
         -0.3175, -0.4011,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 99.09it/s] 


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 026 | Total loss: 9.618 | Reg loss: 0.012 | Tree loss: 9.618 | Accuracy: 0.000000 | 0.887 sec/iter
Epoch: 00 | Batch: 001 / 026 | Total loss: 9.614 | Reg loss: 0.011 | Tree loss: 9.614 | Accuracy: 0.000000 | 0.824 sec/iter
Epoch: 00 | Batch: 002 / 026 | Total loss: 9.612 | Reg loss: 0.010 | Tree loss: 9.612 | Accuracy: 0.000000 | 0.808 sec/iter
Epoch: 00 | Batch: 003 / 026 | Total loss: 9.608 | Reg loss: 0.009 | Tree loss: 9.608 | Accuracy: 0.000000 | 0.798 sec/iter
Epoch: 00 | Batch: 004 / 026 | Total loss: 9.605 | Reg loss: 0.009 | Tree loss: 9.605 | Accuracy: 0.000000 | 0.792 sec/iter
Epoch: 00 | Batch: 005 / 026 | Total loss: 9.602 | Reg loss: 0.008 | Tree loss: 9.602 | Accuracy: 0.000000 | 0.79 sec/iter
Epoch: 00 | Batch: 006 / 026 | Total loss: 9.598 | Reg loss: 0.008 | Tree loss: 9.598 | Accuracy: 0.000000 | 0.788 s

Epoch: 02 | Batch: 009 / 026 | Total loss: 9.509 | Reg loss: 0.008 | Tree loss: 9.509 | Accuracy: 0.136719 | 0.827 sec/iter
Epoch: 02 | Batch: 010 / 026 | Total loss: 9.508 | Reg loss: 0.008 | Tree loss: 9.508 | Accuracy: 0.123047 | 0.827 sec/iter
Epoch: 02 | Batch: 011 / 026 | Total loss: 9.501 | Reg loss: 0.008 | Tree loss: 9.501 | Accuracy: 0.181641 | 0.827 sec/iter
Epoch: 02 | Batch: 012 / 026 | Total loss: 9.503 | Reg loss: 0.009 | Tree loss: 9.503 | Accuracy: 0.150391 | 0.827 sec/iter
Epoch: 02 | Batch: 013 / 026 | Total loss: 9.497 | Reg loss: 0.009 | Tree loss: 9.497 | Accuracy: 0.181641 | 0.827 sec/iter
Epoch: 02 | Batch: 014 / 026 | Total loss: 9.494 | Reg loss: 0.009 | Tree loss: 9.494 | Accuracy: 0.156250 | 0.827 sec/iter
Epoch: 02 | Batch: 015 / 026 | Total loss: 9.491 | Reg loss: 0.010 | Tree loss: 9.491 | Accuracy: 0.132812 | 0.826 sec/iter
Epoch: 02 | Batch: 016 / 026 | Total loss: 9.490 | Reg loss: 0.010 | Tree loss: 9.490 | Accuracy: 0.152344 | 0.826 sec/iter
Epoch: 0

Epoch: 04 | Batch: 019 / 026 | Total loss: 9.335 | Reg loss: 0.016 | Tree loss: 9.335 | Accuracy: 0.130859 | 0.839 sec/iter
Epoch: 04 | Batch: 020 / 026 | Total loss: 9.322 | Reg loss: 0.016 | Tree loss: 9.322 | Accuracy: 0.154297 | 0.839 sec/iter
Epoch: 04 | Batch: 021 / 026 | Total loss: 9.312 | Reg loss: 0.017 | Tree loss: 9.312 | Accuracy: 0.154297 | 0.839 sec/iter
Epoch: 04 | Batch: 022 / 026 | Total loss: 9.302 | Reg loss: 0.017 | Tree loss: 9.302 | Accuracy: 0.154297 | 0.838 sec/iter
Epoch: 04 | Batch: 023 / 026 | Total loss: 9.292 | Reg loss: 0.018 | Tree loss: 9.292 | Accuracy: 0.162109 | 0.838 sec/iter
Epoch: 04 | Batch: 024 / 026 | Total loss: 9.287 | Reg loss: 0.018 | Tree loss: 9.287 | Accuracy: 0.150391 | 0.838 sec/iter
Epoch: 04 | Batch: 025 / 026 | Total loss: 9.235 | Reg loss: 0.018 | Tree loss: 9.235 | Accuracy: 0.210526 | 0.838 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 

Epoch: 07 | Batch: 001 / 026 | Total loss: 9.075 | Reg loss: 0.017 | Tree loss: 9.075 | Accuracy: 0.123047 | 0.852 sec/iter
Epoch: 07 | Batch: 002 / 026 | Total loss: 9.068 | Reg loss: 0.017 | Tree loss: 9.068 | Accuracy: 0.134766 | 0.851 sec/iter
Epoch: 07 | Batch: 003 / 026 | Total loss: 9.035 | Reg loss: 0.017 | Tree loss: 9.035 | Accuracy: 0.160156 | 0.851 sec/iter
Epoch: 07 | Batch: 004 / 026 | Total loss: 9.012 | Reg loss: 0.018 | Tree loss: 9.012 | Accuracy: 0.167969 | 0.851 sec/iter
Epoch: 07 | Batch: 005 / 026 | Total loss: 8.997 | Reg loss: 0.018 | Tree loss: 8.997 | Accuracy: 0.140625 | 0.852 sec/iter
Epoch: 07 | Batch: 006 / 026 | Total loss: 8.985 | Reg loss: 0.018 | Tree loss: 8.985 | Accuracy: 0.152344 | 0.852 sec/iter
Epoch: 07 | Batch: 007 / 026 | Total loss: 8.958 | Reg loss: 0.018 | Tree loss: 8.958 | Accuracy: 0.150391 | 0.852 sec/iter
Epoch: 07 | Batch: 008 / 026 | Total loss: 8.948 | Reg loss: 0.018 | Tree loss: 8.948 | Accuracy: 0.125000 | 0.852 sec/iter
Epoch: 0

Epoch: 09 | Batch: 011 / 026 | Total loss: 8.390 | Reg loss: 0.022 | Tree loss: 8.390 | Accuracy: 0.119141 | 0.854 sec/iter
Epoch: 09 | Batch: 012 / 026 | Total loss: 8.336 | Reg loss: 0.022 | Tree loss: 8.336 | Accuracy: 0.123047 | 0.854 sec/iter
Epoch: 09 | Batch: 013 / 026 | Total loss: 8.322 | Reg loss: 0.022 | Tree loss: 8.322 | Accuracy: 0.152344 | 0.854 sec/iter
Epoch: 09 | Batch: 014 / 026 | Total loss: 8.319 | Reg loss: 0.022 | Tree loss: 8.319 | Accuracy: 0.117188 | 0.854 sec/iter
Epoch: 09 | Batch: 015 / 026 | Total loss: 8.288 | Reg loss: 0.022 | Tree loss: 8.288 | Accuracy: 0.154297 | 0.854 sec/iter
Epoch: 09 | Batch: 016 / 026 | Total loss: 8.255 | Reg loss: 0.023 | Tree loss: 8.255 | Accuracy: 0.136719 | 0.854 sec/iter
Epoch: 09 | Batch: 017 / 026 | Total loss: 8.270 | Reg loss: 0.023 | Tree loss: 8.270 | Accuracy: 0.123047 | 0.854 sec/iter
Epoch: 09 | Batch: 018 / 026 | Total loss: 8.241 | Reg loss: 0.023 | Tree loss: 8.241 | Accuracy: 0.128906 | 0.854 sec/iter
Epoch: 0

...

Epoch: 89 | Batch: 016 / 026 | Total loss: 2.928 | Reg loss: 0.029 | Tree loss: 2.928 | Accuracy: 0.136719 | 0.844 sec/iter
Epoch: 89 | Batch: 017 / 026 | Total loss: 2.944 | Reg loss: 0.029 | Tree loss: 2.944 | Accuracy: 0.154297 | 0.844 sec/iter
Epoch: 89 | Batch: 018 / 026 | Total loss: 2.887 | Reg loss: 0.029 | Tree loss: 2.887 | Accuracy: 0.177734 | 0.844 sec/iter
Epoch: 89 | Batch: 019 / 026 | Total loss: 2.840 | Reg loss: 0.029 | Tree loss: 2.840 | Accuracy: 0.175781 | 0.844 sec/iter
Epoch: 89 | Batch: 020 / 026 | Total loss: 2.950 | Reg loss: 0.029 | Tree loss: 2.950 | Accuracy: 0.134766 | 0.844 sec/iter
Epoch: 89 | Batch: 021 / 026 | Total loss: 2.875 | Reg loss: 0.029 | Tree loss: 2.875 | Accuracy: 0.185547 | 0.844 sec/iter
Epoch: 89 | Batch: 022 / 026 | Total loss: 2.945 | Reg loss: 0.029 | Tree loss: 2.945 | Accuracy: 0.140625 | 0.844 sec/iter
Epoch: 89 | Batch: 023 / 026 | Total loss: 2.917 | Reg loss: 0.029 | Tree loss: 2.917 | Accuracy: 0.115234 | 0.844 sec/iter
Epoch: 8

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 92 | Batch: 000 / 026 | Total loss: 3.022 | Reg loss: 0.028 | Tree loss: 3.022 | Accuracy: 0.138672 | 0.843 sec/iter
Epoch: 92 | Batch: 001 / 026 | Total loss: 3.036 | Reg loss: 0.028 | Tree loss: 3.036 | Accuracy: 0.146484 | 0.842 sec/iter
Epoch: 92 | Batch: 002 / 026 | Total loss: 3.011 | Reg loss: 0.028 | Tree loss: 3.011 | Accuracy: 0.115234 | 0.842 sec/iter
Epoch: 92 | Batch: 003 / 026 | Total loss: 2.959 | Reg loss: 0.028 | Tree loss: 2.959 | Accuracy: 0.185547 | 0.842 sec/iter
Epoch: 92 | Batch: 004 / 026 | Total loss: 3.092 | Reg loss: 0.028 | Tree loss: 3.092 | Accuracy: 0.121094 | 0.842 sec/iter
Epoch: 92 | Batch: 005 / 026 | Total loss: 3.053 | Reg loss: 0.028 | Tree loss: 3.053 | Ac

Epoch: 94 | Batch: 008 / 026 | Total loss: 2.970 | Reg loss: 0.029 | Tree loss: 2.970 | Accuracy: 0.136719 | 0.841 sec/iter
Epoch: 94 | Batch: 009 / 026 | Total loss: 2.943 | Reg loss: 0.029 | Tree loss: 2.943 | Accuracy: 0.146484 | 0.841 sec/iter
Epoch: 94 | Batch: 010 / 026 | Total loss: 2.912 | Reg loss: 0.029 | Tree loss: 2.912 | Accuracy: 0.134766 | 0.841 sec/iter
Epoch: 94 | Batch: 011 / 026 | Total loss: 3.013 | Reg loss: 0.029 | Tree loss: 3.013 | Accuracy: 0.128906 | 0.841 sec/iter
Epoch: 94 | Batch: 012 / 026 | Total loss: 2.931 | Reg loss: 0.029 | Tree loss: 2.931 | Accuracy: 0.136719 | 0.841 sec/iter
Epoch: 94 | Batch: 013 / 026 | Total loss: 2.998 | Reg loss: 0.029 | Tree loss: 2.998 | Accuracy: 0.146484 | 0.841 sec/iter
Epoch: 94 | Batch: 014 / 026 | Total loss: 2.860 | Reg loss: 0.029 | Tree loss: 2.860 | Accuracy: 0.173828 | 0.841 sec/iter
Epoch: 94 | Batch: 015 / 026 | Total loss: 2.945 | Reg loss: 0.029 | Tree loss: 2.945 | Accuracy: 0.154297 | 0.841 sec/iter
Epoch: 9

Epoch: 96 | Batch: 018 / 026 | Total loss: 2.937 | Reg loss: 0.029 | Tree loss: 2.937 | Accuracy: 0.152344 | 0.84 sec/iter
Epoch: 96 | Batch: 019 / 026 | Total loss: 2.963 | Reg loss: 0.029 | Tree loss: 2.963 | Accuracy: 0.142578 | 0.84 sec/iter
Epoch: 96 | Batch: 020 / 026 | Total loss: 2.893 | Reg loss: 0.029 | Tree loss: 2.893 | Accuracy: 0.152344 | 0.84 sec/iter
Epoch: 96 | Batch: 021 / 026 | Total loss: 2.948 | Reg loss: 0.029 | Tree loss: 2.948 | Accuracy: 0.132812 | 0.84 sec/iter
Epoch: 96 | Batch: 022 / 026 | Total loss: 2.890 | Reg loss: 0.029 | Tree loss: 2.890 | Accuracy: 0.138672 | 0.84 sec/iter
Epoch: 96 | Batch: 023 / 026 | Total loss: 2.899 | Reg loss: 0.029 | Tree loss: 2.899 | Accuracy: 0.158203 | 0.84 sec/iter
Epoch: 96 | Batch: 024 / 026 | Total loss: 2.859 | Reg loss: 0.029 | Tree loss: 2.859 | Accuracy: 0.144531 | 0.84 sec/iter
Epoch: 96 | Batch: 025 / 026 | Total loss: 2.801 | Reg loss: 0.029 | Tree loss: 2.801 | Accuracy: 0.157895 | 0.839 sec/iter
Average sparsen

Epoch: 99 | Batch: 000 / 026 | Total loss: 2.998 | Reg loss: 0.028 | Tree loss: 2.998 | Accuracy: 0.154297 | 0.839 sec/iter
Epoch: 99 | Batch: 001 / 026 | Total loss: 3.049 | Reg loss: 0.028 | Tree loss: 3.049 | Accuracy: 0.138672 | 0.838 sec/iter
Epoch: 99 | Batch: 002 / 026 | Total loss: 3.094 | Reg loss: 0.028 | Tree loss: 3.094 | Accuracy: 0.152344 | 0.838 sec/iter
Epoch: 99 | Batch: 003 / 026 | Total loss: 3.040 | Reg loss: 0.028 | Tree loss: 3.040 | Accuracy: 0.134766 | 0.838 sec/iter
Epoch: 99 | Batch: 004 / 026 | Total loss: 3.029 | Reg loss: 0.028 | Tree loss: 3.029 | Accuracy: 0.119141 | 0.838 sec/iter
Epoch: 99 | Batch: 005 / 026 | Total loss: 2.975 | Reg loss: 0.028 | Tree loss: 2.975 | Accuracy: 0.162109 | 0.838 sec/iter
Epoch: 99 | Batch: 006 / 026 | Total loss: 2.997 | Reg loss: 0.028 | Tree loss: 2.997 | Accuracy: 0.144531 | 0.838 sec/iter
Epoch: 99 | Batch: 007 / 026 | Total loss: 2.950 | Reg loss: 0.028 | Tree loss: 2.950 | Accuracy: 0.166016 | 0.838 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 9.948453608247423


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 970


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
9675
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Patte

============== Pattern 283 ==============
============== Pattern 284 ==============
============== Pattern 285 ==============
============== Pattern 286 ==============
============== Pattern 287 ==============
============== Pattern 288 ==============
============== Pattern 289 ==============
============== Pattern 290 ==============
============== Pattern 291 ==============
============== Pattern 292 ==============
============== Pattern 293 ==============
============== Pattern 294 ==============
============== Pattern 295 ==============
============== Pattern 296 ==============
============== Pattern 297 ==============
============== Pattern 298 ==============
============== Pattern 299 ==============
============== Pattern 300 ==============
============== Pattern 301 ==============
============== Pattern 302 ==============
============== Pattern 303 ==============
============== Pattern 304 ==============
============== Pattern 305 ==============
============== Pattern 306 =======

============== Pattern 547 ==============
============== Pattern 548 ==============
============== Pattern 549 ==============
============== Pattern 550 ==============
============== Pattern 551 ==============
============== Pattern 552 ==============
============== Pattern 553 ==============
============== Pattern 554 ==============
============== Pattern 555 ==============
============== Pattern 556 ==============
============== Pattern 557 ==============
============== Pattern 558 ==============
============== Pattern 559 ==============
============== Pattern 560 ==============
============== Pattern 561 ==============
============== Pattern 562 ==============
============== Pattern 563 ==============
============== Pattern 564 ==============
============== Pattern 565 ==============
============== Pattern 566 ==============
============== Pattern 567 ==============
============== Pattern 568 ==============
============== Pattern 569 ==============
============== Pattern 570 =======

============== Pattern 841 ==============
============== Pattern 842 ==============
============== Pattern 843 ==============
============== Pattern 844 ==============
============== Pattern 845 ==============
============== Pattern 846 ==============
============== Pattern 847 ==============
============== Pattern 848 ==============
============== Pattern 849 ==============
============== Pattern 850 ==============
============== Pattern 851 ==============
============== Pattern 852 ==============
============== Pattern 853 ==============
============== Pattern 854 ==============
============== Pattern 855 ==============
============== Pattern 856 ==============
============== Pattern 857 ==============
============== Pattern 858 ==============
============== Pattern 859 ==============
============== Pattern 860 ==============
============== Pattern 861 ==============
============== Pattern 862 ==============
============== Pattern 863 ==============
============== Pattern 864 =======